# Neural Net model 1

- V0: HyperOpt NN with one layer
    - Minimum achieved around 15 epochs (test with that)
- V1: very sensitive to random seed, redid gridsearch with new random seed

Scores:
- V0: CV: 2.2372, LB: 2.2428
    - The solver is very sensitive random seed! 
    -{'seed1': 239, 'seed2': 1234, 'layer_2': {'on2': True, 'activation2': 'relu', 'units2': 179.31129328655433, 'dropout2': 0.27147227595028317}, 'optimizer': 'adamax', 'layer_1': {'units1': 141.28833823317106, 'activation1': 'tanh', 'on1': True, 'dropout1': 0.6945806925899082}, 'layer_3': {'activation3': 'relu', 'on3': False, 'units3': 16, 'dropout3': 0.25}, 'n_epoch': 9, 'batch_size': 400}
- V1: CV: LB: (not uploaded directly to Kaggle yet)

In [6]:
import os
import random
import time
import sys
import pickle
import datetime

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

#os.environ['KERAS_BACKEND']='tensorflow'
#os.environ['THEANO_FLAGS'] = 'device=cpu'
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.optimizers import SGD
from keras.callbacks import Callback
from keras.layers.advanced_activations import PReLU

import keras

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

from sklearn.preprocessing import LabelEncoder, StandardScaler
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.cross_validation import StratifiedKFold, train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import log_loss

from ml_toolbox.kaggle import KaggleResult

%matplotlib inline


Using Theano backend.


In [13]:
dir_in = 'data_ori'
dir_feat = 'data'
sub_dir = 'model_1_nn'

description = 'model_1_nn_V2'

In [14]:
feature_files = ['features_brand_model_bag',
                 'features_brand_bag',
                 'features_appid_installed',
                 'features_label_app_installed']

# Function to read feature file
def open_feature_file(fname, samples='train'):
    if fname[-3:] == 'csv':
        if samples=='train':
            X = gatrain[['device_id']].merge( pd.read_csv(os.path.join(dir_feat, fname)), on='device_id', how='left')
        else:
            X = gatest[['device_id']].merge( pd.read_csv(os.path.join(dir_feat, fname)), on='device_id', how='left')
            
        X.drop('device_id', axis=1, inplace=True)
        X.fillna(0, inplace=True)
        
        for c in X.columns:
            if X[c].max()>1:
                X[c] = StandardScaler().fit_transform(X)
                
        return csr_matrix(X.values)
    else:
        # Assume it is a pickle file
        with open(os.path.join(dir_feat, '{}_{}.pickle'.format(fname,samples)), 'rb') as f:
            return pickle.load(f)

## Load train/test data

In [15]:
gatrain = pd.read_csv(os.path.join(dir_in,'gender_age_train.csv'))
gatest = pd.read_csv(os.path.join(dir_in,'gender_age_test.csv'))
targetencoder = LabelEncoder().fit(gatrain.group)
y = targetencoder.transform(gatrain.group)
nclasses = len(targetencoder.classes_)

dummy_y = np_utils.to_categorical(y)

In [16]:
fw = [1, 1, 1, 1]
Xtrain = hstack([open_feature_file(f) for f in feature_files],format='csr')
Xtest = hstack([open_feature_file(f,'test') for f in feature_files],format='csr')

print('Train on {} features'.format(Xtrain.shape[1]))
print('Test on {} features'.format(Xtrain.shape[1]))


Train on 21527 features
Test on 21527 features


In [17]:
# With selection (15xxx features): 2.27427
# Without selection (21527 features): 2.27427
selector = VarianceThreshold().fit(Xtrain)
Xtrain = selector.transform(Xtrain)
Xtest = selector.transform(Xtest)

In [19]:
# Load CV sets
train_cv = pd.read_csv(os.path.join(dir_in, 'gender_age_train_cv.csv'))
test_cv = pd.read_csv(os.path.join(dir_in, 'gender_age_test_cv.csv'))

X_train, X_val = Xtrain[train_cv.sample_nr.values, :], Xtrain[test_cv.sample_nr.values, :]
y_train, y_val = dummy_y[train_cv.sample_nr], dummy_y[test_cv.sample_nr]

## Test Keras model

In [20]:
def batch_generatorp(X, batch_size, shuffle):
    number_of_batches = X.shape[0] / np.ceil(X.shape[0]/batch_size)
    counter = 0
    sample_index = np.arange(X.shape[0])
    while True:
        batch_index = sample_index[batch_size * counter:batch_size * (counter + 1)]
        X_batch = X[batch_index, :].toarray()
        counter += 1
        yield X_batch#, np.ones(X_batch.shape[0])
        if (counter == number_of_batches):
            counter = 0
            
def nn_model(n_feat, params):
    model = Sequential()
    model.add(Dense(params['layer_1']['units1'], 
                    input_dim=n_feat, 
                    init='normal', 
                    activation=params['layer_1']['activation1']))

    model.add(Dropout(params['layer_1']['dropout1']))

    if params['layer_2']['on2']:
        model.add(Dense(params['layer_2']['units2'], 
                        init='normal', 
                        activation=params['layer_2']['activation2']))
        #https://www.kaggle.com/poiss0nriot/talkingdata-mobile-user-demographics/bag-of-apps-keras-11-08-16-no-val/run/328610
        #model.add(PReLU())
        model.add(Dropout(params['layer_2']['dropout2']))

    model.add(Dense(12, init='normal', activation='softmax'))    

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=params['optimizer'], metrics=['accuracy'])  #logloss
    
    return model

def score_nn(X_train, X_val, y_train, y_val, params, description):
    # create model
    print ('Model with following parameters: %s' % (params))
    
    np.random.seed(params['seed'])
    random.seed(params['seed']) # not sure if needed or not

    model = nn_model(X_train.shape[1], params)

    hist_all = prediction_check('test', os.path.join(sub_dir, description + '_model.h5'))

    fit=model.fit(X_train.todense(), y_train, nb_epoch = int(params['n_epoch']), batch_size=int(params['batch_size']),
                             validation_data=(X_val.todense(), y_val), callbacks=[hist_all], verbose=2) 
    
    # evaluate the model
    pred_val = model.predict_generator(generator=batch_generatorp(X_val, 32, False), val_samples=X_val.shape[0])

    cv_score = log_loss(y_val, pred_val)

    print('logloss val {}'.format(cv_score))
    
    return cv_score, model, pred_val, hist_all.filepath

class prediction_check(Callback):
    def __init__(self, tag='', filepath=''):
        self.predhist = pd.DataFrame(columns=['acc','logloss','val_acc','val_logloss'])
        self.tag = str(tag)
        self.counter = 0
        self.best = 10.0
        self.best_rounds = 1
        self.wait = 0
        self.patience = 3
        self.filepath = filepath
        
    def on_epoch_end(self, epoch, logs={}):
        #print logs.values()
        self.counter += 1
        self.predhist.loc[epoch] = logs.values()
        
        current_score = logs['val_loss']
        
        if current_score < self.best:
            self.best = current_score
            self.best_train = logs['loss']
            self.best_rounds = self.counter
            self.wait = 0
            self.model.save_weights(self.filepath, overwrite=True)
        else:
            if self.wait >= self.patience:
                self.model.stop_training = True
                self.model.load_weights(self.filepath)
                print('Best number of rounds: %d \n Val loss: %f \n' % (self.best_rounds, self.best))
            self.wait += 1
            
    def on_train_end(self, logs={}):
        self.model.load_weights(self.filepath)
        self.model.save(self.filepath, overwrite=True)

In [21]:
params = {'optimizer': 'adam',
          'batch_size': 32,
          'n_epoch': 4,
         'layer_1': {'on1': True,
                     'units1': 150,
                     'activation1': 'relu',
                     'dropout1': 0.6},
         'layer_2': {'on2': False,
                    'units2': 150,
                    'activation2': 'tanh',
                    'dropout2': 0.2},
          'layer_3': {'on3': False,
                     'units3': 12,
                     'activation3': 'sigmoid',
                     'dropout3': 0.2}
          }
# According: 
#https://www.kaggle.com/agavranis/talkingdata-mobile-user-demographics/bag-of-apps-keras-11-08-16-no-val/code
params = {'optimizer': 'adadelta',
          'batch_size': 400,
          'n_epoch': 16,
         'layer_1': {'on1': True,
                     'units1': 150,
                     'activation1': 'relu',
                     'dropout1': 0.4},
         'layer_2': {'on2': True,
                    'units2': 50,
                    'activation2': 'relu',
                    'dropout2': 0.2},
          'layer_3': {'on3': False,
                     'units3': 12,
                     'activation3': 'sigmoid',
                     'dropout3': 0.2}
          }

# After 80 rounds op hyperopt evals:
params = {'optimizer': 'adamax',
          'batch_size': 400,
          'n_epoch': 17,
          'seed': 1234,
         'layer_1': {'on1': True,
                     'units1': 221,
                     'activation1': 'tanh',
                     'dropout1': 0.84},
         'layer_2': {'on2': True,
                    'units2': 232,
                    'activation2': 'relu',
                    'dropout2': 0.23},
          'layer_3': {'on3': False,
                     'units3': 12,
                     'activation3': 'sigmoid',
                     'dropout3': 0.2}
          }

# V1
params = {'type': 'keras_nn',
          'n_epoch': 50, # 11.406643551001192, 
          'optimizer': 'adadelta', 
          'seed': 89171,
          'layer_1': {'units1': 400, 
                      'activation1': 'tanh', 
                      'on1': True, 
                      'dropout1': 0.8},
          'layer_2': {'on2': True, 
                      'activation2': 'relu', 
                      'units2': 130, 
                      'dropout2': 0.35}, 
          'batch_size': 400}

In [24]:
models_out = []
scores = []

for s in np.random.randint(99999,size=5):
    params['seed'] = s
    score, model, pred_val,modelfile = score_nn(X_train, X_val, y_train, y_val, params, description + '_' + str(s))
    
    print('Validation score:',score)

    model_out = {'model': modelfile,
                 'score': score,
                 'params': params}
    
    models_out.append(model_out)
    scores.append(score)

Model with following parameters: {'optimizer': 'adadelta', 'layer_1': {'units1': 400, 'activation1': 'tanh', 'on1': True, 'dropout1': 0.8}, 'layer_2': {'on2': True, 'activation2': 'relu', 'units2': 130, 'dropout2': 0.35}, 'batch_size': 400, 'seed': 53207, 'n_epoch': 50, 'type': 'keras_nn'}
Train on 67229 samples, validate on 7416 samples
Epoch 1/50
32s - loss: 2.4423 - acc: 0.1280 - val_loss: 2.3907 - val_acc: 0.1593
Epoch 2/50
29s - loss: 2.3787 - acc: 0.1543 - val_loss: 2.3366 - val_acc: 0.1761
Epoch 3/50
27s - loss: 2.3442 - acc: 0.1665 - val_loss: 2.3121 - val_acc: 0.1824
Epoch 4/50
26s - loss: 2.3260 - acc: 0.1746 - val_loss: 2.2983 - val_acc: 0.1838
Epoch 5/50
26s - loss: 2.3114 - acc: 0.1791 - val_loss: 2.2883 - val_acc: 0.1900
Epoch 6/50
26s - loss: 2.3017 - acc: 0.1832 - val_loss: 2.2829 - val_acc: 0.1907
Epoch 7/50
25s - loss: 2.2948 - acc: 0.1873 - val_loss: 2.2842 - val_acc: 0.1904
Epoch 8/50
26s - loss: 2.2884 - acc: 0.1879 - val_loss: 2.2798 - val_acc: 0.1897
Epoch 9/50
2

In [25]:
outputfile = 'models_nn_1_V2_{}_{:.4f}_{:.4f}.pickle'.format(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M"),
                                                                cv_score,
                                                                -1.)
output = {'script': 'train_model1_nn',
          'model_params': params,
          'no_models': 1,
          'cross_validation': {'type': 'gender_age_train_cv.csv'},
          'models': models_out}


with open(os.path.join(sub_dir, outputfile), 'wb') as f:
    pickle.dump(output,f)

In [26]:
print ('Model with following parameters: %s' % (params))

np.random.seed(params['seed1'])
random.seed(params['seed1']) # not sure if needed or not

X_train, X_val, y_train, y_val = train_test_split(Xtrain, dummy_y, stratify=y,
                                                  test_size=0.1, random_state=params['seed2'])

model = Sequential()
model.add(Dense(params['layer_1']['units1'], 
                input_dim=X_train.shape[1], 
                init='normal', 
                activation=params['layer_1']['activation1']))

model.add(Dropout(params['layer_1']['dropout1']))

if params['layer_2']['on2']:
    model.add(Dense(params['layer_2']['units2'], 
                    init='normal', 
                    activation=params['layer_2']['activation2']))
    #https://www.kaggle.com/poiss0nriot/talkingdata-mobile-user-demographics/bag-of-apps-keras-11-08-16-no-val/run/328610
    #model.add(PReLU())
    model.add(Dropout(params['layer_2']['dropout2']))

if params['layer_3']['on3']:
    model.add(Dense(params['layer_3']['units3'], 
                    init='normal', 
                    activation=params['layer_3']['activation3']))

    model.add(Dropout(params['layer_3']['dropout3']))

model.add(Dense(12, init='normal', activation='softmax'))   
# Compile model
model.compile(loss='categorical_crossentropy', optimizer=params['optimizer'], metrics=['accuracy'])

fit=model.fit(X_train.todense(), y_train, nb_epoch = params['n_epoch'] , batch_size=400,
                         validation_data=(X_val.todense(), y_val), verbose=2) 

Model with following parameters: {'seed1': 239, 'seed2': 1234, 'layer_2': {'on2': True, 'activation2': 'relu', 'units2': 179.31129328655433, 'dropout2': 0.27147227595028317}, 'optimizer': 'adamax', 'layer_1': {'units1': 141.28833823317106, 'activation1': 'tanh', 'on1': True, 'dropout1': 0.6945806925899082}, 'layer_3': {'activation3': 'relu', 'on3': False, 'units3': 16, 'dropout3': 0.25}, 'n_epoch': 9, 'batch_size': 400}
Train on 67179 samples, validate on 7466 samples
Epoch 1/9
18s - loss: 2.3804 - acc: 0.1563 - val_loss: 2.3042 - val_acc: 0.1846
Epoch 2/9
15s - loss: 2.3067 - acc: 0.1805 - val_loss: 2.2756 - val_acc: 0.1956
Epoch 3/9
13s - loss: 2.2816 - acc: 0.1921 - val_loss: 2.2623 - val_acc: 0.2025
Epoch 4/9
13s - loss: 2.2646 - acc: 0.1978 - val_loss: 2.2538 - val_acc: 0.2045
Epoch 5/9
13s - loss: 2.2537 - acc: 0.2034 - val_loss: 2.2471 - val_acc: 0.2056
Epoch 6/9
14s - loss: 2.2425 - acc: 0.2078 - val_loss: 2.2448 - val_acc: 0.2061
Epoch 7/9
16s - loss: 2.2350 - acc: 0.2105 - va

In [30]:
# evaluate the model
scores_val = model.predict_generator(generator=batch_generatorp(X_val, 32, False), val_samples=X_val.shape[0])

cv_score = log_loss(y_val, scores_val)

print('logloss val {}'.format(log_loss(y_val, scores_val)))

logloss val 2.23722598618


## Create test score

In [27]:
# evaluate the model
pred_test = model.predict_generator(generator=batch_generatorp(Xtest, 32, False), val_samples=Xtest.shape[0])
pred = pd.DataFrame(pred_test, index = gatest['device_id'], columns=targetencoder.classes_)

In [28]:
kag = KaggleResult(pred.values, pred.index.values, cv_score=cv_score, description='NN Model 1 - with seeds', subdir=sub_dir)

In [29]:
if kag.validate():
    kag.upload()
print kag.lb_score

2.2428



## Development

### Build a good cross validation set

In [9]:
train = pd.read_csv(os.path.join(dir_in,'gender_age_train.csv'), dtype={'device_id': np.str})
train['sample_nr'] = train.index
test = pd.read_csv(os.path.join(dir_in,'gender_age_test.csv'), dtype={'device_id': np.str})
events = pd.read_csv(os.path.join(dir_in,'events.csv'), dtype={'device_id': np.str})
unique_device_id = events.device_id.unique()

index = train.device_id.isin(unique_device_id)
index2 = [not i for i in index]
train1 = train[index].sort_values(by='device_id').reset_index(drop=True)
train2 = train[index2].sort_values(by='device_id').reset_index(drop=True)

train1, test1 = train_test_split(train1, test_size=0.100, random_state=1974,     stratify=train1['group'].values)
train2, test2 = train_test_split(train2, test_size=0.099, random_state=1974, stratify=train2['group'].values)
train = pd.concat([train1, train2], ignore_index=True).sort_values(by='device_id').reset_index(drop=True)
test = pd.concat([test1, test2], ignore_index=True).sort_values(by='device_id').reset_index(drop=True)

In [10]:
test.head()

,device_id,gender,age,group,sample_nr
0,-1001337759327042486,M,30,M29-31,39902
1,-1007249684892521754,M,40,M39+,7310
2,-1008484851265061759,M,29,M29-31,65975
3,-1014599235541061307,M,30,M29-31,27471
4,-1021073753429555564,M,36,M32-38,67125


In [11]:
X_train, X_val = Xtrain.tocsc()[train.sample_nr, :], Xtrain.tocsc()[test.sample_nr, :]
y_train, y_val = dummy_y[train.sample_nr], dummy_y[test.sample_nr]

In [12]:
np.random.randint(99999, size=5)

array([42782, 44836,  3898, 38776, 92747])

In [13]:
class prediction_check(Callback):
    def __init__(self, tag='', filepath='./'):
        super(Callback, self).__init__()
        
        self.predhist = pd.DataFrame(columns=['acc','logloss','val_acc','val_logloss'])
        self.tag = str(tag)
        self.counter = 0
        self.best = 10.0
        self.best_rounds = 1
        self.wait = 0
        self.patience = 3
        self.filepath = filepath
        
    def on_epoch_end(self, epoch, logs={}):
        #print logs.values()
        self.counter += 1
        self.predhist.loc[epoch] = logs.values()
        
        current_score = logs['val_loss']
        
        if current_score < self.best:
            self.best = current_score
            self.best_train = logs['loss']
            self.best_rounds = self.counter
            self.wait = 0
            self.model.save_weights(self.filepath, overwrite=True)
        else:
            if self.wait >= self.patience:
                self.model.stop_training = True
                self.model.load_weights(self.filepath)
                print('Best number of rounds: %d \n Val loss: %f \n' % (self.best_rounds, self.best))
            self.wait += 1

In [14]:
for s in [89171]: #np.random.randint(99999, size=5):
    np.random.seed(s)
    random.seed(s) # not
    
    

    # create model
    print ('Model with following parameters: %s and seed %d' % (params, s))

    model = Sequential()
    model.add(Dense(params['layer_1']['units1'], 
                    input_dim=X_train.shape[1], 
                    init='normal', 
                    activation=params['layer_1']['activation1']))

    model.add(Dropout(params['layer_1']['dropout1']))

    if params['layer_2']['on2']:
        model.add(Dense(params['layer_2']['units2'], 
                        init='normal', 
                        activation=params['layer_2']['activation2']))
        #https://www.kaggle.com/poiss0nriot/talkingdata-mobile-user-demographics/bag-of-apps-keras-11-08-16-no-val/run/328610
        #model.add(PReLU())
        model.add(Dropout(params['layer_2']['dropout2']))

    if params['layer_3']['on3']:
        model.add(Dense(params['layer_3']['units3'], 
                        init='normal', 
                        activation=params['layer_3']['activation3']))

        model.add(Dropout(params['layer_3']['dropout3']))

    model.add(Dense(12, init='normal', activation='softmax'))    

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=params['optimizer'], metrics=['accuracy'])  #logloss

    hist_all = prediction_check('test', filepath=sub_dir + '/test.h5' )

    fit=model.fit(X_train.todense(), y_train, nb_epoch = 20, batch_size=int(params['batch_size']),
                             validation_data=(X_val.todense(), y_val), verbose=2,
                 callbacks=[hist_all]) 

    # evaluate the model
    scores_val = model.predict_generator(generator=batch_generatorp(X_val, 32, False), val_samples=X_val.shape[0])

    cv_score = log_loss(y_val, scores_val)

    print('logloss val {}'.format(log_loss(y_val, scores_val)))


Model with following parameters: {'optimizer': 'adamax', 'layer_1': {'units1': 141.28833823317106, 'activation1': 'tanh', 'on1': True, 'dropout1': 0.6945806925899082}, 'layer_3': {'activation3': 'relu', 'on3': False, 'units3': 16, 'dropout3': 0.25}, 'layer_2': {'on2': True, 'activation2': 'relu', 'units2': 179.31129328655433, 'dropout2': 0.27147227595028317}, 'batch_size': 400, 'seed1': 239, 'n_epoch': 9} and seed 89171
Train on 67229 samples, validate on 7416 samples
Epoch 1/20
17s - loss: 2.3863 - acc: 0.1531 - val_loss: 2.3097 - val_acc: 0.1806
Epoch 2/20
13s - loss: 2.3065 - acc: 0.1826 - val_loss: 2.2770 - val_acc: 0.1923
Epoch 3/20
13s - loss: 2.2812 - acc: 0.1926 - val_loss: 2.2655 - val_acc: 0.1986
Epoch 4/20
13s - loss: 2.2656 - acc: 0.1994 - val_loss: 2.2596 - val_acc: 0.1967
Epoch 5/20
14s - loss: 2.2532 - acc: 0.2032 - val_loss: 2.2548 - val_acc: 0.2027
Epoch 6/20
14s - loss: 2.2432 - acc: 0.2087 - val_loss: 2.2558 - val_acc: 0.1988
Epoch 7/20
15s - loss: 2.2330 - acc: 0.21

In [16]:

scores_val = model.predict_generator(generator=batch_generatorp(X_val, 32, False), val_samples=X_val.shape[0])

cv_score = log_loss(y_val, scores_val)
print cv_score

2.24941839454


In [17]:
# evaluate the model
pred_test = model.predict_generator(generator=batch_generatorp(Xtest, 32, False), val_samples=Xtest.shape[0])
pred = pd.DataFrame(pred_test, index = gatest['device_id'], columns=targetencoder.classes_)

In [18]:
kag = KaggleResult(pred.values, pred.index.values, cv_score=cv_score, description='NN Model 1 - random test - new CV set', subdir=sub_dir)

In [ ]:
if kag.validate():
    kag.upload()
print kag.lb_score

### Build convo network

In [34]:
from keras.layers import Convolution1D, MaxPooling1D, Flatten, Reshape, Embedding
# set parameters:


max_features = 2000
maxlen = X_train.shape[1]
batch_size = 32
embedding_dims = 50
nb_filter = 250
filter_length = 3
hidden_dims = 250
nb_epoch = 3

model = Sequential()

model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen,
                    dropout=0.2))

# we add a Convolution1D, which will learn nb_filter
# word group filters of size filter_length:
model.add(Convolution1D(nb_filter=nb_filter,
                        filter_length=filter_length,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1))
# we use max pooling:
model.add(MaxPooling1D(pool_length=model.output_shape[1]))

# We flatten the output of the conv layer,
# so that we can add a vanilla dense layer:
model.add(Flatten())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))
# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(12, init='normal', activation='softmax'))    

# Compile model
model.compile(loss='categorical_crossentropy', optimizer=params['optimizer'], metrics=['accuracy'])  #logloss


